In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
import itertools
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from collections import Counter
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn import neural_network
from sklearn import linear_model
import os
from textblob import TextBlob
import lightgbm as lgbm
import nltk
from embeddings import GloveEmbedding, FastTextEmbedding, KazumaCharEmbedding, ConcatEmbedding

# from multilabel_data_handler import get_data

ModuleNotFoundError: No module named 'catboost'

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# file used to write preserve the results of the classfier
# confusion matrix and precision recall fscore matrix

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    return plt

In [4]:
##saving the classification report
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='macro'))
    avg.append(accuracy_score(y_true, y_pred, normalize=True))
    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support','accuracy']
    list_all=list(metrics_summary)
    list_all.append(cm.diagonal())
    class_report_df = pd.DataFrame(
        list_all,
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-2] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

In [5]:
from commen_preprocess import *

.....start_cleaning.........
<hashtag> britain exit <hashtag> rape refugee


In [32]:
eng_train_dataset = pd.read_csv('../Data/english_dataset/english_dataset_added_features.tsv', sep=',')
#hindi_train_dataset = pd.read_csv('../Data/hindi_dataset/hindi_dataset.tsv', sep='\t',header=None)
german_train_dataset = pd.read_csv('../Data/german_dataset/german_dataset_added_features.tsv', sep=',')
eng_train_dataset=eng_train_dataset.drop(['Unnamed: 0'], axis=1)
german_train_dataset=german_train_dataset.drop(['Unnamed: 0'], axis=1)


In [34]:
eng_train_dataset.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE


In [7]:
# eng_train_dataset = pd.read_csv('../AMI@EVALITA2018/en_training.tsv', sep='\t')
# eng_test_dataset = pd.read_csv('../AMI@EVALITA2018/en_testing.tsv', sep='\t')
eng_train_dataset = pd.read_csv('../sem_eval_data/train_en.tsv', sep='\t')
eng_dev_dataset = pd.read_csv('../sem_eval_data/dev_en.tsv', sep='\t')
eng_test_dataset = pd.read_csv('../sem_eval_data/test_en.tsv', sep='\t')
eng_dev_own_dataset = pd.read_csv('../sem_eval_data/101-200 test_en.tsv', sep='\t')


# AMI_train_dataset =pd.read_csv('../sem_eval_data/AMI_train.tsv',sep='\t')
# AMI_train_dataset.head()
# AMI_train_dataset=AMI_train_dataset.drop(['misogyny_category'],axis=1)
# AMI_train_dataset.columns = ['id', 'text','HS','TR']
# AMI_train_dataset['AG'] = 0
# dict1 = {"active": 1, "passive": 0}
# AMI_train_dataset=AMI_train_dataset.replace({"TR":dict1})
# AMI_train_dataset.head()
# frames = [eng_train_dataset,AMI_train_dataset]
# eng_train_dataset = pd.concat(frames)

eng_train_dataset = eng_train_dataset.sample(frac=1).reset_index(drop=True)


In [11]:
eng_dev_own_dataset=eng_dev_own_dataset[0:199]

In [12]:
eng_dev_own_dataset['HS'].value_counts()

1.0    124
0.0     75
Name: HS, dtype: int64

In [13]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import os
GLOVE_MODEL_FILE1= "../../LEAM-master/glove.twitter.27B/glove.twitter.27B.200d.txt"
GLOVE_MODEL_FILE2="../../embeddings/glove.840B.300d.txt"
GLOVE_MODEL_FILE3="../../embeddings/wiki.en.vec"




import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    i=0
    for line in f:
        i=i+1
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
        if(i%10000==0):
            print("count"+str(i))
    print("Done.",len(model)," words loaded!")
    return model



EMBEDDING_DIM = 200


# word2vec_model = KeyedVectors.load_word2vec_format(tmp_file)
word2vec_model1 = loadGloveModel(GLOVE_MODEL_FILE1)
word2vec_model2 = loadGloveModel(GLOVE_MODEL_FILE2)
word2vec_model3 =  KeyedVectors.load_word2vec_format(GLOVE_MODEL_FILE3)

Loading Glove Model
count10000
count20000
count30000
count40000
count50000
count60000
count70000
count80000
count90000
count100000
count110000
count120000
count130000
count140000
count150000
count160000
count170000
count180000
count190000
count200000
count210000
count220000
count230000
count240000
count250000
count260000
count270000
count280000
count290000
count300000
count310000
count320000
count330000
count340000
count350000
count360000
count370000
count380000
count390000
count400000
count410000
count420000
count430000
count440000
count450000
count460000
count470000
count480000
count490000
count500000
count510000
count520000
count530000
count540000
count550000
count560000
count570000
count580000
count590000
count600000
count610000
count620000
count630000
count640000
count650000
count660000
count670000
count680000
count690000
count700000
count710000
count720000
count730000
count740000
count750000
count760000
count770000
count780000
count790000
count800000
count810000
count820000
count

In [14]:
from gensim.models import Word2Vec
word2vec_model4 = Word2Vec.load("../../embeddings/word2vec.model")

In [15]:
### stopwords and punctuations are not removed but text is cleaned and stemmed
def glove_tokenize_norem(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    words = text.split()
    words =[ps.stem(word) for word in words]
    return words

####stopwords and punctuations are removed along with that text is cleaned ans stemmed
def glove_tokenize(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    words =[ps.stem(word) for word in words]
    return words

def pos_tokenize(text):
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    tags = nltk.pos_tag(text.split())
    tag_list = [x[1] for x in tags]
    return tag_list


def glove_tokenize_embed(text):
    #text = tokenizer(text)
    text=clean(text,for_embeddings=True, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    return words

def glove_tokenize_vocab(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    words = text.split()
    return words


In [16]:
text="#hastag i am awesome"
pos_tokenize(text)

['NN', 'VBZ', 'VBN', 'NN', 'VBP', 'JJ']

In [17]:
def convert_class_label(input_text):
    if input_text==1:
        return 'misogyny'
    elif input_text==0:
        return 'non-misogyny'
    else:
        print('Wrong Input', input_text)
        sys.exit(0)

In [18]:
# pd_train = pd.DataFrame(columns=['id','misogynous','text'])
eng_train_dataset["text"].replace('', np.nan, inplace=True)
eng_train_dataset.dropna(subset=['text'], inplace=True)

pd_train_binary = eng_train_dataset[['id','text','HS']]
pd_dev_binary = eng_dev_dataset[['id','text','HS']]
pd_dev_own_binary = eng_dev_own_dataset[['id','text','HS']]

# pd_train_binary = eng_train_dataset[['id','misogynous','text','misogyny_category','target','male','female']]
# pd_train_category = eng_train_dataset[['id','misogynous','text','misogyny_category']]
# pd_train_target = eng_train_dataset[['id','misogynous','text','target']]
pd_test = eng_test_dataset[['id','text']]

# pd_train_category = pd_train_category.loc[pd_train_category['misogynous'] == 1]
# pd_train_target = pd_train_target.loc[pd_train_target['misogynous'] == 1]
# pd_train_target.drop(['misogynous'], axis=1)                                      
# pd_train_category.drop(['misogynous'], axis=1)                                      

# # pd_train['class'] =pd_train.apply(lambda row: convert_class_label(row['misogynous']), axis=1)

pd_train_binary['class'] = pd_train_binary['HS']
pd_dev_binary['class'] = pd_dev_binary['HS']
pd_dev_own_binary['class'] = pd_dev_own_binary['HS']

# pd_train_category['class'] = pd_train_category['misogyny_category']
# pd_train_target['class'] = pd_train_target['target']

# for count, each in enumerate(train_data):
#     try:
#         pd_train.loc[count]  = [each['id'], convert_class_label(each['CounterSpeech']), each['Community'],each['Category'],each['commentText']]
#     except:
#         pass
print('Training Data Loading Completed...')

/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training Data Loading Completed...


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
pd_dev_own_binary

,id,text,HS,class
0,31568,"Bro is a bitch, fucking cunt https://t.co/dQ8J...",1.0,1.0
1,30734,I'm rewatching Breaking Bad & I could see why ...,1.0,1.0
2,31227,@charliekirk11 @asia742 364 down only about 12...,1.0,1.0
3,31224,>Harasses women and calls them bitch and crabb...,1.0,1.0
4,31139,"Bitch you ain't gotta call my phone, matter fa...",1.0,1.0
5,34076,@ABCPolitics A illegal aliens showing exactly ...,1.0,1.0
6,32479,Bitch....how the fuck. Siri you a dumb ass hoe...,1.0,1.0
7,32985,#FactCheck true #TuesdayThoughts #WalkAwayFrom...,1.0,1.0
8,30284,@GMA @TVMarci His own fault #SENDTHEMBACK,1.0,1.0
9,30072,@christianllamar @LindseyGrahamSC stop w 'we h...,0.0,0.0


In [20]:
def get_data(pd_train):
    comments=pd_train['text'].values
    labels=pd_train['class'].values
    list_comment=[]
    for comment,label in zip(comments,labels):
        temp={}
        temp['text']=comment
        temp['label']=label
        list_comment.append(temp)
    return list_comment 

def get_data_test(pd_test):
    comments=pd_test['text'].values
    list_comment=[]
    for comment in comments:
        temp={}
        temp['text']=comment
        list_comment.append(temp)
    return list_comment 


In [21]:
vocab={}
def create_vocab(data):
    comments = get_data(data)
    for comment in comments:
        words=glove_tokenize_vocab(comment['text'])
        for word in words:
            if word in vocab.keys():
                vocab[word]=vocab[word]+1 
            else:
                vocab[word]=1
                
        

In [22]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=12,
                       allow_soft_placement=True, device_count = {'CPU': 12})

def get_embeddings(messages):
      
    with tf.Session(config=config) as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_emb = session.run(embed(messages))
            
    print("ending")
    return np.array(message_emb)

/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_11
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_12:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/v

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

In [23]:
from nltk.stem import PorterStemmer
from pathlib import Path

    # file exists
ps = PorterStemmer()

TOKENIZER = glove_tokenize

#google encoding used where text is cleaned  
def gen_data_google(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        #X.append(tokenizer(comment['text']))
        X.append(clean(comment['text'], remove_stopwords=True, remove_punctuations=True))
    #TFIDF_feature = 'bpe_text'

    #Word Level Features
    X =get_embeddings(X)
    # print y
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

#google encoding used where text is not cleaned 
def gen_data_google2(data):
    comments = get_data(data)
    X, y = [],[]
    for comment in comments:
        y.append(comment['label'])
        X.append(clean(comment['text'], remove_stopwords=False, remove_punctuations=False))
    #Word Level Features
    X =get_embeddings(X)
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X,y

###get data of subjectivity
def subject(data):
    comments = get_data(data)
    X = []
    for comment in comments:
        comment=clean(comment['text'], remove_stopwords=False, remove_punctuations=False)
        X.append(TextBlob(comment).sentiment.subjectivity)
        #Word Level Features
    
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X



### tfidf feature generation was used here where stopwords and punctuations are removed 
def gen_data_new_tfidf(data):
        comments = get_data(data)
        comments_test=get_data_test(pd_test)
        comments_dev=get_data_test(pd_dev_binary)
        
        X, y = [], []
        for comment in comments:
            y.append(comment['label'])
            X.append(comment['text'])
       
        X1=[]
        for comment in comments_test:
            X1.append(comment['text'])
        
    
        for comment in comments_dev:
            X1.append(comment['text'])
        
        
        word_to_vec_file = Path("tfidf_word_vectorizer.pk")
        char_to_vec_file = Path("tfidf_char_vectorizer.pk")
        if word_to_vec_file.is_file() :
            with open('tfidf_word_vectorizer.pk', 'rb') as fout:
                word_vectorizer=pickle.load(fout)
        else:
            word_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                    min_df=1, 
                    strip_accents='unicode',
                    #smooth_idf=1,
                    analyzer='word', 
                    stop_words='english',
                    tokenizer=TOKENIZER,             
                    max_features=5000)
            word_vectorizer.fit(X+X1)
            with open('tfidf_word_vectorizer.pk', 'wb') as fout:
               pickle.dump(word_vectorizer,fout)



        if  char_to_vec_file.is_file():
            with open('tfidf_char_vectorizer.pk', 'rb') as fout:
                 char_vectorizer=pickle.load(fout)
        else:
           char_vectorizer = TfidfVectorizer(
                            sublinear_tf=True,
                            strip_accents='unicode',
                            analyzer='char',
                            #stop_words='english',
                            ngram_range=(2, 6),
                            max_features=10000)
           char_vectorizer.fit(X+X1)
           with open('tfidf_char_vectorizer.pk', 'wb') as fout:
               pickle.dump(char_vectorizer,fout)
        
        pos_to_vec_file = Path("tfidf_pos_vectorizer.pk")
    
        if  pos_to_vec_file.is_file():
            with open('tfidf_pos_vectorizer.pk', 'rb') as fout:
                 pos_vectorizer=pickle.load(fout)
        else:
           #Word Level Features
           pos_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                    min_df=1, 
                    strip_accents='unicode',
                    smooth_idf=1,
                    analyzer='word', 
                    stop_words=None,
                    tokenizer=pos_tokenize,             
                    max_features=5000)


           pos_vectorizer.fit(X+X1)
           with open('tfidf_pos_vectorizer.pk', 'wb') as fout:
             pickle.dump(pos_vectorizer,fout)


    
    
        
        
        test_pos_features = pos_vectorizer.transform(X)
        test_word_features = word_vectorizer.transform(X)
        test_char_features = char_vectorizer.transform(X)
        X = list(hstack([test_char_features, test_word_features,test_pos_features]).toarray())
        #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
        return X, y






### tfidf feature generation was used here where stopwords and punctuations are not removed 
def gen_data_new_tfidf2(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                stop_words=None,
                tokenizer=glove_tokenize_norem,             
                max_features=5000)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words=None,
    ngram_range=(2, 6),
    max_features=10000)
    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    with open('tfidf_word_vectorize_noclean.pk', 'wb') as fout:
         pickle.dump(word_vectorizer,fout)

    with open('tfidf_char_vectorizer_noclean.pk', 'wb') as fout:
         pickle.dump(char_vectorizer,fout)
        
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_rem(data):
    X,_=gen_data_google(data)
    X1,y=gen_data_new_tfidf(data)
#     X1,y=gen_data_old_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

## combination of cleaned google encodings and tfidf features where stopwords and punctuations are ssremoved 
def combine_tf_google_norem(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf2(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def combine_tf_rem_google_norem(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def combine_tf_norem_google_rem(data):
    X,_=gen_data_google(data)
    X1,y=gen_data_new_tfidf2(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def gen_data_embed(data,word2vec,embed_dim):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        words = glove_tokenize_embed(comment['text'].lower())
        emb = np.zeros(embed_dim)
        for word in words:
            try:
                emb += word2vec[word]
            except:
                pass
        if len(words)!=0:
            emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    return X, y






def combine_tf_rem_google_norem_embed(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf(data)
    print("1loaded ")
    X2,_=gen_data_embed(data,word2vec_model3,300)
    print("2loaded")
    X3,_=gen_data_embed(data,word2vec_model2,300)
    print("3loaded")
    X4,_=gen_data_embed(data,word2vec_model1,200)
    print("4loaded")
    X5=  subject(data)
    print("5loaded")
    X6,_=gen_data_embed(data,word2vec_model4,300)
    print("6loaded")
    #X=np.concatenate((np.array(X), np.array(X1),np.array(X2),np.array(X3).reshape(len(X3),1),), axis=1)
    X=np.concatenate((np.array(X), np.array(X1),np.array(X2),np.array(X3),np.array(X4),np.array(X5).reshape(len(X5),1),np.array(X6)), axis=1)
    
    return X,y



###old tfidf

def gen_data_old_tfidf(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])
    with open('../tfidf_word_vectorizer.pk', 'rb') as fin:
        word_vectorizer = pickle.load(fin)

    with open('../tfidf_char_vectorizer.pk', 'rb') as fin:
        char_vectorizer = pickle.load(fin)


    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

def select_comments_whose_embedding_exists(flag):
    # selects the comments as in mean_glove_embedding method
    # Processing
    comments = get_data(flag)
    X, Y = [], []
    comment_return = []
    for tweet in comments:
        #print(tweet)
        _emb = 0
        words = TOKENIZER(tweet['text'].lower())
        for w in words:
            #print(w)
            if w in word2vec_model and w is not None:  # Check if embeeding there in GLove model
                _emb+=1
        if _emb:   # Not a blank tweet
            comment_return.append(tweet)
    print('Comments selected:', len(comment_return))
    return comment_return

def gen_data():
    comments = select_comments_whose_embedding_exists(0)
    X, y = [], []
    for comment in comments:
        words = glove_tokenize(comment['text'].lower())
        emb = numpy.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    # print y
    y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    print
    return X, y

## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_glove_rem():
    X,_=gen_data_google()
    X1,y=gen_data_new_tfidf()
#     X1,y=gen_data_old_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

In [24]:
SCALE_POS_WEIGHT=None

In [25]:
# from imblearn.combine import SMOTETomek 
from sklearn.metrics import accuracy_score
import joblib
from sklearn.model_selection import StratifiedKFold as skf
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedShuffleSplit

def binny_classifier_run(X,y,model,model_name,label_map,img_name,report_name,save_model=False):
    Classifier_Train_X = np.array(X, copy=False)
    Classifier_Train_Y = y
    temp=[]
    for data in Classifier_Train_Y:
        temp.append(label_map[data])
    SCALE_POS_WEIGHT=temp.count(0)/temp.count(1)
    
    Classifier_Train_Y=np.array(temp)
    
    
    #model_featureSelection = SelectFromModel(ensemble.RandomForestClassifier(n_estimators=500, class_weight='balanced',n_jobs=12, max_depth=7))
    #model_featureSelection = SelectFromModel(ExtraTreesClassifier(n_estimators=50))
    model = LogisticRegression()
    model_featureSelection = RFE(model,1300,step=0.5) 
    
    print('Before Num features=',Classifier_Train_X.shape[1], Counter(Classifier_Train_Y))
    Classifier_Train_X = model_featureSelection.fit_transform(Classifier_Train_X,Classifier_Train_Y)
    print('After Num features=',Classifier_Train_X.shape[1])

    
    if(save_model==True):
        Classifier=model
        Classifier.fit(Classifier_Train_X, Classifier_Train_Y)
        filename = model_name+'_task_1.joblib.pkl'
        joblib.dump(Classifier, filename, compress=9)
        filename1 = 'select_features_task1.joblib.pkl'
        joblib.dump(model_featureSelection, filename1, compress=9)
    else:
        kf = skf(n_splits=10,shuffle=True)
        y_total_preds=[] 
        y_total=[]
        count=0

        for train_index, test_index in kf.split(Classifier_Train_X,Classifier_Train_Y):
            X_train, X_test = Classifier_Train_X[train_index], Classifier_Train_X[test_index]
            y_train, y_test = Classifier_Train_Y[train_index], Classifier_Train_Y[test_index]

            classifier=model 
            classifier.fit(X_train,y_train)
            y_preds = classifier.predict(X_test)
            for ele in y_test:
                y_total.append(ele)
            for ele in y_preds:
                y_total_preds.append(ele)
            y_pred_train = classifier.predict(X_train)
            count=count+1       
            print('accuracy_train:',accuracy_score(y_train, y_pred_train),'accuracy_test:',accuracy_score(y_test, y_preds))
            print('TRAINING:')
            print(classification_report( y_train, y_pred_train ))
            print("TESTING:")
            print(classification_report( y_test, y_preds ))
            
        report = classification_report( y_total, y_total_preds )
        cm=confusion_matrix(y_total, y_total_preds)
        plt=plot_confusion_matrix(cm,normalize= True,target_names = ['non_hatespeech','hatespeech'],title = "Confusion Matrix")
        import time
        import zipfile

        timestr = time.strftime("%Y%m%d-%H%M%S")

        plt.savefig('task1_'+timestr+'_'+model_name+'_'+img_name)
        print(model)
        print(report)
        prRandomForestClassifierint(accuracy_score(y_total, y_total_preds))
        df_result=pandas_classification_report(y_total,y_total_preds)
        df_result.to_csv('task1_'+timestr+'__'+model_name+'_'+report_name,  sep=',')

        
        
        
def classifier_run(X,y,X_dev,y_dev,X_dev_own,y_dev_own,model,model_name,label_map,img_name,report_name,save_model=False):
    Classifier_Train_X = np.array(X, copy=False)
    Classifier_Train_Y = y
    Classifier_Dev_X = np.array(X_dev, copy=False)
    Classifier_Dev_Y = y_dev
    X_dev_own = np.array(X_dev_own, copy=False)
    
    temp=[]
    for data in Classifier_Train_Y:
        temp.append(label_map[data])
    SCALE_POS_WEIGHT=temp.count(0)/temp.count(1)
    print(SCALE_POS_WEIGHT)
    Classifier_Train_Y=np.array(temp)
    
    temp=[]
    for data in Classifier_Dev_Y:
        temp.append(label_map[data])
    Classifier_Dev_Y=np.array(temp)
    
    temp=[]
    for data in y_dev_own:
        try:
            temp.append(int(data))
        except:
            temp.append(0)
    y_dev_own=np.array(temp)
    
    
    
    model_featureSelection = SelectFromModel(ensemble.RandomForestClassifier(n_estimators=50, class_weight='balanced',n_jobs=12, max_depth=6,random_state=42))
    print(model_featureSelection)
    
    print('Before Num features=',Classifier_Train_X.shape[1], Counter(Classifier_Train_Y))
    Classifier_Train_X = model_featureSelection.fit_transform(Classifier_Train_X,Classifier_Train_Y)
    print('After Num features=',Classifier_Train_X.shape[1])
    print('Before Num features=',Classifier_Dev_X.shape[1], Counter(Classifier_Dev_Y))
    Classifier_Dev_X = model_featureSelection.transform(Classifier_Dev_X)
    print('After Num features=',Classifier_Dev_X.shape[1])
    print('Before Num features=',X_dev_own.shape[1], Counter(y_dev_own))
    X_dev_own = model_featureSelection.transform(X_dev_own)
    print('After Num features=',X_dev_own.shape[1])

    
    if(save_model==True):
        Classifier=model
        X=np.concatenate((np.array(Classifier_Train_X), np.array(Classifier_Dev_X)), axis=0)
        Y=np.concatenate((np.array(Classifier_Train_Y), np.array(Classifier_Dev_Y)), axis=0)
        
        Classifier.fit(X,Y)
        filename = model_name+'_task_1.joblib.pkl'
        joblib.dump(Classifier, filename, compress=9)
        filename1 = model_name+'select_features_task1.joblib.pkl'
        joblib.dump(model_featureSelection, filename1, compress=9)
    else:
        X=np.concatenate((np.array(Classifier_Train_X), np.array(Classifier_Dev_X)), axis=0)
        Y=np.concatenate((np.array(Classifier_Train_Y), np.array(Classifier_Dev_Y)), axis=0)
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
        for train_index, test_index in sss.split(X, Y):
                print("TRAIN:", train_index, "TEST:", test_index)
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = Y[train_index], Y[test_index]
        
        model.fit(X_train,y_train)
        y_preds = model.predict(X_test) 
        y_pred_train = model.predict(X_train)
        y_pred_own = model.predict(X_dev_own) 
        
        print('accuracy_train:',accuracy_score(y_train, y_pred_train),'accuracy_test:',accuracy_score(y_test, y_preds),'accuracy_test_own:',accuracy_score(y_dev_own, y_pred_own))
        print('TRAINING:')
        print(classification_report( y_train, y_pred_train ))
        print("TESTING:")
        print(classification_report( y_test, y_preds ))
        print("TESTING_OWN:")
        print(classification_report( y_dev_own, y_pred_own ))
                

In [26]:
classifier_model = 'gaussian'
feature_model = 'google_nopreprocess_tfidf_preprocess_embed'
img_name = 'cm.png'
report_name = 'report.csv'

In [29]:
data_name= 'pd_train_binary'

# X,y=get_feature(f_type=feature_model,data=pd_train_binary)

# X_dev,y_dev=get_feature(f_type=feature_model,data=pd_dev_binary)
    
X_dev_own,y_dev_own=get_feature(f_type=feature_model,data=pd_dev_own_binary)
    
label_map = {
             1: 1,
             0: 0
                 }


ending


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['fifti'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


1loaded 
2loaded
3loaded
4loaded
5loaded
6loaded
google_nopreprocess_tfidf_preprocess_embed


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/ipykernel/__main__.py:225: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [31]:
X_dev_own.shape

(199, 21613)

In [32]:
# np.save('X_train.npy', X)
# np.save('X_dev.npy', X_dev)
np.save('X_dev_own.npy', X_dev_own)
# np.save('y_train.npy', y)
# np.save('y_dev.npy', y_dev)
np.save('y_dev_own.npy', y_dev_own)


In [ ]:
model=get_model(m_type=classifier_model)

In [35]:
from imblearn.over_sampling import ADASYN
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from nltk.classify.scikitlearn import SklearnClassifier

SCALE_POS_WEIGHT=None

def get_model(m_type=None):
    if not m_type:
        print("ERROR: Please specify a model type!")
        return None
    if m_type == 'decision_tree_classifier':
        logreg = tree.DecisionTreeClassifier()
    elif m_type == 'gaussian':
        logreg = GaussianNB()
    elif m_type == 'logistic_regression':
        logreg = LogisticRegression(n_jobs=10, random_state=42)
    elif m_type == 'MLPClassifier':
#         logreg = neural_network.MLPClassifier((500))
        logreg = neural_network.MLPClassifier((100),random_state=42,early_stopping=True)
    elif m_type == 'KNeighborsClassifier':
#         logreg = neighbors.KNeighborsClassifier(n_neighbors = 10)
        logreg = neighbors.KNeighborsClassifier()
    elif m_type == 'ExtraTreeClassifier':
        logreg = tree.ExtraTreeClassifier()
    elif m_type == 'ExtraTreeClassifier_2':
        logreg = ensemble.ExtraTreesClassifier()
    elif m_type == 'RandomForestClassifier':
        logreg = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=7)
    elif m_type == 'SVC':
        #logreg = LinearSVC(dual=False,max_iter=200)
        logreg = SVC(kernel='linear',random_state=1526)
        
    elif m_type == 'Catboost':
        logreg = CatBoostClassifier(iterations=1000,learning_rate=0.2,l2_leaf_reg=500,depth=10,use_best_model=False, random_state=42,scale_pos_weight=SCALE_POS_WEIGHT)
#         logreg = CatBoostClassifier(scale_pos_weight=0.8, random_seed=42,);
    elif m_type == 'XGB_classifier':
#         logreg=XGBClassifier(silent=False,eta=0.1,objective='binary:logistic',max_depth=5,min_child_weight=0,gamma=0.2,subsample=0.8, colsample_bytree = 0.8,scale_pos_weight=1,n_estimators=500,reg_lambda=3,nthread=12)
        logreg=XGBClassifier(silent=False,objective='binary:logistic',scale_pos_weight=0.8,reg_lambda=3,nthread=12, random_state=42)
    elif m_type == 'binny_test':
        clf1 = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=6,max_features='auto')
        clf2 = tree.DecisionTreeClassifier(random_state=42, class_weight='balanced',max_depth=6)
        clf3 = LogisticRegression(class_weight='balanced',penalty="l2",C=0.1, dual=True, random_state=42, n_jobs=3)
        clf4 = XGBClassifier(silent=False,objective='binary:logistic',scale_pos_weight=0.8,reg_lambda=3,nthread=12, random_state=42)
        est_list = [('lr', clf1), ('rf', clf2), ('gnb', clf3), ('xgb', clf4)]
        logreg = ensemble.VotingClassifier(est_list,voting='soft',n_jobs=6)
    elif m_type == 'ensem':
        ens_list=[]
        model= LGBMClassifier(objective='binary',max_depth=7,learning_rate=0.2,num_leaves=50,boosting_type='gbdt',metric='binary_logloss',random_state=5,reg_lambda=20,silent=True,scale_pos_weight=SCALE_POS_WEIGHT)
        ens_list.append(('lgb1',model))
        model1 = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=5)
        ens_list.append(('rf',model1)) 
        model2 = neural_network.MLPClassifier((100),random_state=42,early_stopping=True)
        ens_list.append(('mlp',model2)) 
        model3 = SVC(kernel='linear',probability=True,random_state=2018)
        ens_list.append(('svc',model3)) 
        model4 = SVC(kernel='linear',probability=True,random_state=1526)
        ens_list.append(('svc1',model4)) 
#         model5 = SVC(kernel='linear',probability=True,random_state=2019)
#         ens_list.append(('svc2',model5)) 
        
        logreg = ensemble.VotingClassifier(ens_list,voting='hard',n_jobs=6)
        
    elif m_type == 'light_gbm':
        logreg = LGBMClassifier(objective='binary',max_depth=3,learning_rate=0.2,num_leaves=20,boosting_type='gbdt',metric='binary_logloss',random_state=5,reg_lambda=20,silent=False)

    else:
        print("give correct model")
    print(logreg)
    return logreg

def get_feature(f_type=None,data=None):
    if not f_type:
        print("ERROR: Please specify a model type!")
        return None
    if f_type == 'google_not_preprocess':
        X,y=gen_data_google2(data)
    elif f_type == 'word_to_vec_embed':
        X,y=gen_data_embed(data)
    elif f_type == 'google_preprocess':
        X,y=gen_data_google(data)
    elif f_type == 'tfidf_not_preprocess':
        X,y=gen_data_new_tfidf2(data)
    elif f_type == 'tfidf_preprocess':
        X,y=gen_data_new_tfidf(data)
    elif f_type == 'google_preprocess_tfidf_preprocess':
        X,y=combine_tf_google_rem(data)
    elif f_type == 'google_nopreprocess_tfidf_nopreprocess':
        X,y=combine_tf_google_norem(data)
    elif f_type == 'google_preprocess_tfidf_nopreprocess':
        X,y=combine_tf_norem_google_rem(data)
    elif f_type == 'google_nopreprocess_tfidf_preprocess':
        X,y=combine_tf_rem_google_norem(data)
    elif f_type == 'google_nopreprocess_tfidf_preprocess_embed':
        X,y=combine_tf_rem_google_norem_embed(data)
    else:
        print("give correct feature selection")    
    print(f_type)
    return X,y



ModuleNotFoundError: No module named 'imblearn'

In [ ]:
#list_of_model = ['decision_tree_classifier', 'gaussian', 'logistic_regression', 'MLPClassifier', 'RandomForestClassifier',
#                 'SVC', 'Catboost', 'XGB_classifier']
list_of_model = ['SVC']
#list_of_model = ['logistic_regression']
label_map = {
             1: 1,
             0: 0
                 }
img_name = 'cm.png'
report_name = 'report.csv'

for each_model in list_of_model:
    model=get_model(m_type=each_model)
    y=np.load('y_train.npy')
    y_dev=np.load('y_dev.npy')
    y_dev_own=np.load('y_dev_own.npy')
    X=np.load('X_train.npy')
    X_dev=np.load('X_dev.npy')
    X_dev_own=np.load('X_dev_own.npy')
    classifier_run(X,y,X_dev,y_dev,X_dev_own,y_dev_own,model,each_model,label_map,img_name,report_name,save_model=False)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=1526,
  shrinking=True, tol=0.001, verbose=False)
1.3790642347343378
SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=6, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=12, oob_score=False, random_state=42,
            verbose=0, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=None)
Before Num features= 21613 Counter({0: 5217, 1: 3783})
After Num features= 1342
Before Num features= 21613 Counter({0: 573, 1: 427})
After Num features= 1342
Before Num features= 21613 Counter({1: 124, 0: 75})
After Num f

In [ ]:
y_dev_own.shape

In [ ]:
with open('tfidf_word_vectorizer.pk', 'rb') as fout:
         word_vectorizer=pickle.load(fout)

with open('tfidf_char_vectorizer.pk', 'rb') as fout:
        char_vectorizer=pickle.load(fout)
clf_task1=joblib.load('Catboost_task_1.joblib.pkl')
select_task1=joblib.load('select_features_task1.joblib.pkl')


In [ ]:
#### adding pesudo labelling 
from nltk.stem import PorterStemmer
ps = PorterStemmer()

TOKENIZER = glove_tokenize


#google encoding used where text is not cleaned 
def gen_data_google_test():
    comments = get_data_test(pd_test)
    X=[]
    for comment in comments:
        X.append(clean(comment['text'], remove_stopwords=False, remove_punctuations=False))
    #Word Level Features
    X =get_embeddings(X)
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X

### tfidf feature generation was used here where stopwords and punctuations are removed 
def gen_data_new_tfidf_test():
    comments_test=get_data_test(pd_test)
    X=[]
    for comment in comments_test:
        X.append(comment['text'])
      
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X


def gen_data_embed_test():
    comments = get_data_test(pd_test)
    X=[]
    for comment in comments:
        words = glove_tokenize(comment['text'].lower())
        emb = np.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        if len(words)!=0:
            emb /= len(words)
        X.append(emb)
        
    return X

def subject_test():
    comments = get_data_test(pd_test)
    X = []
    for comment in comments:
        comment=clean(comment['text'], remove_stopwords=False, remove_punctuations=False)
        X.append(TextBlob(comment).sentiment.subjectivity)
        #Word Level Features
    
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X


def combine_tf_rem_google_norem_embed_test():
    X=gen_data_google_test()
    X1=gen_data_new_tfidf_test()
    X2=gen_data_embed_test()
    X3=subject_test()
    X=np.concatenate((np.array(X), np.array(X1),np.array(X2),np.array(X3).reshape(len(X3),1)), axis=1)
    return X





In [ ]:
X_test=combine_tf_rem_google_norem_embed_test()
list_1=[]
for x in X_test:
    x=x.reshape(1,-1)
    temp=x
    temp=select_task1.transform(temp)
    predict1=clf_task1.predict(temp)
    list_1.append(int(predict1[0]))


In [ ]:
temp_2=y+list_1
temp=np.concatenate((X,X_test),axis=0)
print(len(temp_2))

In [ ]:
def binny_classifier_run_pl(X,y,X_test_pl,y_test_pl,model,model_name,label_map,img_name,report_name,save_model=False):
    Classifier_Train_X = np.array(X, copy=False)
    Classifier_Train_Y = y
    temp=[]
    for data in Classifier_Train_Y:
        temp.append(label_map[data])
    SCALE_POS_WEIGHT=temp.count(0)/temp.count(1)
    
    Classifier_Train_Y=np.array(temp)
    
    
    print('Before Num features=',Classifier_Train_X.shape[1], Counter(Classifier_Train_Y))
    Classifier_Train_X = select_task1.transform(Classifier_Train_X)
    X_test_pl = select_task1.transform(X_test_pl)
    print('After Num features=',Classifier_Train_X.shape[1])

    
    if(save_model==True):
        Classifier=model
        Classifier.fit(Classifier_Train_X, Classifier_Train_Y)
        filename = model_name+'_task_1.joblib.pkl'
        joblib.dump(Classifier, filename, compress=9)
        #filename1 = 'select_features_task1.joblib.pkl'
        #joblib.dump(model_featureSelection, filename1, compress=9)
    else:
        kf = skf(n_splits=10,shuffle=True)
        y_total_preds=[] 
        y_total=[]
        count=0

        for train_index, test_index in kf.split(Classifier_Train_X,Classifier_Train_Y):
    #         print('cv_fold',count)
    #         print(train_index, test_index)
    #         print(type(Classifier_Train_X), type(Classifier_Train_Y))
            X_train, X_test = Classifier_Train_X[train_index], Classifier_Train_X[test_index]
            y_train, y_test = Classifier_Train_Y[train_index], Classifier_Train_Y[test_index]

        #     model_featureSelection = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=0.1, dual=True, random_state=42, n_jobs=1))
            
        #     ada = ADASYN(random_state=42, ratio='minority')
        #     ada = SMOTETomek(random_state=42,n_jobs=4)
        #     X_train, y_train = ada.fit_sample(X_train, y_train)
        #     print('After OVERSAMPLING Num poinst=', Counter(y_train))
            classifier=model 
            y_pl=np.concatenate((y_train,y_test_pl),axis=0)
            X_pl=np.concatenate((X_train,X_test_pl),axis=0)

            classifier.fit(X_pl,y_pl)
            y_preds = classifier.predict(X_test)
            for ele in y_test:
                y_total.append(ele)
            for ele in y_preds:
                y_total_preds.append(ele)
            y_pred_train = classifier.predict(X_train)
            count=count+1       
            print('accuracy_train:',accuracy_score(y_train, y_pred_train),'accuracy_test:',accuracy_score(y_test, y_preds))
            print('TRAINING:')
            print(classification_report( y_train, y_pred_train ))
            print("TESTING:")
            print(classification_report( y_test, y_preds ))
            
        report = classification_report( y_total, y_total_preds )
        cm=confusion_matrix(y_total, y_total_preds)
        plt=plot_confusion_matrix(cm,normalize= True,target_names = ['non_hatespeech','hatespeech'],title = "Confusion Matrix")
        import time
        import zipfile

        timestr = time.strftime("%Y%m%d-%H%M%S")

        plt.savefig('task1_'+timestr+'_'+model_name+'_'+img_name)
        print(model)
        print(report)
        print(accuracy_score(y_total, y_total_preds))
        df_result=pandas_classification_report(y_total,y_total_preds)
        df_result.to_csv('task1_'+timestr+'__'+model_name+'_'+report_name,  sep=',')


In [ ]:
list_of_model = ['Catboost']
#list_of_model = ['logistic_regression']


for each_model in list_of_model:
    model=get_model(m_type=each_model)
    binny_classifier_run_pl(X,y,X_test,list_1,model,each_model,label_map,img_name,report_name,save_model=True)


In [ ]:
len(y)

In [ ]:
len(list_1)